In [2]:
# ---------------------------------------
# IMPORTS
# ---------------------------------------
import time
import csv
from typing import Dict, Any, List
import requests

In [3]:
# ---------------------------------------
# CONFIGURACIÓN LAST.FM
# ---------------------------------------
BASE_URL = "https://ws.audioscrobbler.com/2.0/"
API_KEY = "3bd1dc669ba0fd4ee507954de7dd51a0"

In [4]:
# ---------------------------------------
# FUNCIONES AUXILIARES
# ---------------------------------------
def normalizar_nombre_artista(nombre: str) -> str:
    """
    Normaliza el nombre del artista para comparar con Spotify.
    Pediste: strip + lower + casefold
    """
    if not nombre:
        return ""
    return nombre.strip().lower().casefold()


def lastfm_request(method: str, params: Dict[str, Any], api_key: str = API_KEY, timeout: int = 30) -> Dict[str, Any]:
    """
    Función genérica para llamar a la API de Last.fm y devolver el JSON como diccionario.

    - method: nombre del método/endpoint de last.fm (ej: 'artist.getInfo')
    - params: parámetros específicos del método (ej: {'artist': 'Coldplay'})
    - api_key: la clave de la API
    - timeout: segundos máximos que esperamos antes de cancelar la petición
    """

    # Construimos todos los parámetros que necesita last.fm en una sola variable
    full_params = {
        "method": method,     # Qué método vamos a usar (ej: artist.getInfo)
        "api_key": api_key,   # Nuestra API Key
        "format": "json",     # Queremos la respuesta en formato JSON
        **params              # Aquí metemos el resto de parámetros específicos (artist, autocorrect, etc.)
    }

    # Hacemos la petición GET a la URL base, pasando los parámetros
    r = requests.get(BASE_URL, params=full_params, timeout=timeout)

    # Si el servidor responde con error HTTP (404, 401, 500...), aquí salta una excepción
    r.raise_for_status()

    # Convertimos la respuesta a JSON (en Python: dict/list)
    data = r.json()

    # A veces last.fm devuelve error "dentro" del JSON aunque el HTTP sea 200 (OK)
    if isinstance(data, dict) and "error" in data:
        raise ValueError(f"Last.fm API error {data.get('error')}: {data.get('message')}")

    # Si todo va bien, devolvemos el JSON
    return data


def leer_artistas_csv(ruta_csv: str) -> List[str]:
    """
    Lee un CSV de UNA sola columna (sin cabecera) y devuelve la lista de artistas.

    Importante:
    - El CSV ya viene normalizado, pero por seguridad volvemos a normalizar.
    - Eliminamos duplicados manteniendo el orden (muy importante para no repetir peticiones).
    """

    artistas = []

    # Abrimos el fichero
    with open(ruta_csv, "r", encoding="utf-8") as f:
        lector = csv.reader(f)

        # Cada fila es una lista, pero como solo hay 1 columna, nos interesa row[0]
        for fila in lector:
            if not fila:
                continue  # por si hay líneas vacías

            nombre = normalizar_nombre_artista(fila[0])
            if nombre:
                artistas.append(nombre)

    # Eliminamos duplicados manteniendo el orden
    # Truco simple: dict.fromkeys mantiene el orden en Python 3.7+
    artistas_sin_duplicados = list(dict.fromkeys(artistas))

    return artistas_sin_duplicados


In [5]:
# ---------------------------------------
# FUNCIÓN PRINCIPAL PARA UN ARTISTA
# ---------------------------------------
def get_artist_info(nombre_artista: str, autocorrect: int = 1, timeout: int = 30) -> Dict[str, Any]:
    """
    Pide a la API de Last.fm información de un artista y devuelve un diccionario
    SIEMPRE con las mismas claves.

    - nombre_artista: nombre del artista (normalizado o no)
    - autocorrect: si vale 1, Last.fm intenta corregir nombres mal escritos
    """

    # 1) Llamamos al endpoint/método "artist.getInfo" de Last.fm
    data = lastfm_request(
        "artist.getInfo",
        {"artist": nombre_artista, "autocorrect": autocorrect},
        timeout=timeout
    )

    # 2) Dentro del JSON, la info principal viene en la clave "artist"
    artist = data.get("artist", {}) or {}

    # 3) Dentro de artist, las estadísticas están dentro de la clave "stats"
    stats = artist.get("stats", {}) or {}

    # 4) Sacamos el nombre “raw” (bonito) y el normalizado para cruzar con Spotify
    nombre_raw = artist.get("name", "") or ""
    nombre_norm = normalizar_nombre_artista(nombre_raw) if nombre_raw else normalizar_nombre_artista(nombre_artista)

    # 5) Ojo: listeners y playcount vienen como texto -> lo pasamos a int
    listeners = int(stats.get("listeners", 0) or 0)
    playcount = int(stats.get("playcount", 0) or 0)

    # 6) Devolvemos un diccionario con lo que os interesa para la tabla de artistas
    return {
        "artist_name_raw": nombre_raw,
        "artist_name_norm": nombre_norm,
        "listeners": listeners,
        "playcount": playcount
    }


In [6]:
# ---------------------------------------
# BUCLE DE PETICIONES (MOVIDO A FUNCIÓN)
# ---------------------------------------
def obtener_info_artistas(
    lista_artistas: List[str],
    pausa_segundos: float = 0.25,
    autocorrect: int = 1,
    timeout: int = 30,
    imprimir_cada: int = 25
) -> List[Dict[str, Any]]:
    """
    Hace el bucle de peticiones a Last.fm para una lista de artistas.

    - lista_artistas: lista ya normalizada y sin duplicados
    - pausa_segundos: pausa entre peticiones para evitar rate limit
    - imprimir_cada: cada cuántos artistas imprimimos progreso (para no saturar la salida)
    """

    lista_resultados = []
    inicio = time.time()

    for idx, artista in enumerate(lista_artistas, start=1):
        try:
            info = get_artist_info(artista, autocorrect=autocorrect, timeout=timeout)
            lista_resultados.append(info)

        except Exception as e:
            # Si falla un artista, lo mostramos y seguimos con el siguiente
            print(f"❌ Error con '{artista}': {e}")
            continue

        # Pausa para no saturar la API
        time.sleep(pausa_segundos)

        # Progreso cada X artistas
        if imprimir_cada and idx % imprimir_cada == 0:
            transcurrido = time.time() - inicio
            print(f"✅ {idx}/{len(lista_artistas)} completados | {transcurrido:.0f}s")

    return lista_resultados


def guardar_resultados_csv(
    resultados: List[Dict[str, Any]],
    ruta_salida: str,
    id_genre: int
) -> None:
    """
    Guarda la lista de diccionarios en CSV.
    Columnas: artist_name_raw, artist_name_norm, listeners, playcount, id_genre
    """

    cabeceras = ["artist_name_raw", "artist_name_norm", "listeners", "playcount", "id_genre"]

    with open(ruta_salida, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=cabeceras)
        writer.writeheader()

        for artista in resultados:
            writer.writerow({
                "artist_name_raw": artista["artist_name_raw"],
                "artist_name_norm": artista["artist_name_norm"],
                "listeners": artista["listeners"],
                "playcount": artista["playcount"],
                "id_genre": id_genre
            })



In [ ]:
# ---------------------------------------
# EXTRACCION POP
# ---------------------------------------
# 1) Leemos el CSV de Spotify (una columna, sin cabecera)
#    -> devuelve lista normalizada y sin duplicados
ruta_csv_pop = "Spotify_artistas_pop.csv"
lista_pop = leer_artistas_csv(ruta_csv_pop)

print("Artistas POP (sin duplicados):", len(lista_pop))

# 2) Pedimos a Last.fm listeners y playcount para cada artista
resultados_pop = obtener_info_artistas(
    lista_pop,
    pausa_segundos=0.25,
    autocorrect=1,
    timeout=30,
    imprimir_cada=25
)

# 3) Guardamos el resultado en un CSV final (para no repetir llamadas)
guardar_resultados_csv(resultados_pop, "lastfm_info_artistas_pop.csv", id_genre=1)

# 4) Ver los 5 primeros
resultados_pop[:5]

Artistas POP (sin duplicados): 351
✅ 25/351 completados | 23s
✅ 50/351 completados | 53s
✅ 75/351 completados | 79s
✅ 100/351 completados | 101s
✅ 125/351 completados | 140s
✅ 150/351 completados | 177s
✅ 175/351 completados | 213s
✅ 200/351 completados | 248s
✅ 225/351 completados | 284s
✅ 250/351 completados | 322s
✅ 275/351 completados | 358s
✅ 300/351 completados | 393s
✅ 325/351 completados | 427s
✅ 350/351 completados | 454s


[{'artist_name_raw': 'ATEEZ',
  'artist_name_norm': 'ateez',
  'listeners': 768643,
  'playcount': 201140830},
 {'artist_name_raw': 'Aurora',
  'artist_name_norm': 'aurora',
  'listeners': 1755377,
  'playcount': 85932243},
 {'artist_name_raw': 'Abraham Mateo',
  'artist_name_norm': 'abraham mateo',
  'listeners': 162950,
  'playcount': 2581795},
 {'artist_name_raw': 'Adele',
  'artist_name_norm': 'adele',
  'listeners': 5509829,
  'playcount': 311239170},
 {'artist_name_raw': 'Ado',
  'artist_name_norm': 'ado',
  'listeners': 646802,
  'playcount': 71606131}]

In [ ]:
# ---------------------------------------
# EXTRACCION ROCK
# ---------------------------------------
# 1) Leemos el CSV de Spotify (una columna, sin cabecera)
#    -> devuelve lista normalizada y sin duplicados
ruta_csv_rock = "spotify_artists_rock_2019_2023.csv"
lista_rock = leer_artistas_csv(ruta_csv_rock)

print("Artistas ROCK (sin duplicados):", len(lista_rock))

# 2) Pedimos a Last.fm listeners y playcount para cada artista
resultados_rock = obtener_info_artistas(
    lista_rock,
    pausa_segundos=0.25,
    autocorrect=1,
    timeout=30,
    imprimir_cada=25
)

# 3) Guardamos el resultado en un CSV final (para no repetir llamadas)
guardar_resultados_csv(resultados_rock, "lastfm_info_artistas_rock.csv", id_genre=2)

# 4) Ver los 5 primeros
resultados_rock[:5]

Artistas ROCK (sin duplicados): 470
✅ 25/470 completados | 20s
✅ 50/470 completados | 39s
✅ 75/470 completados | 64s
✅ 100/470 completados | 96s
✅ 125/470 completados | 115s
✅ 150/470 completados | 136s
✅ 175/470 completados | 157s
✅ 200/470 completados | 179s
✅ 225/470 completados | 199s
✅ 250/470 completados | 221s
✅ 275/470 completados | 247s
✅ 300/470 completados | 269s
✅ 325/470 completados | 289s
✅ 350/470 completados | 310s
✅ 375/470 completados | 365s
✅ 400/470 completados | 387s
✅ 425/470 completados | 409s
✅ 450/470 completados | 438s


[{'artist_name_raw': '2 Minutos',
  'artist_name_norm': '2 minutos',
  'listeners': 71336,
  'playcount': 2507334},
 {'artist_name_raw': '24kGoldn',
  'artist_name_norm': '24kgoldn',
  'listeners': 1164983,
  'playcount': 22573449},
 {'artist_name_raw': '60 juno',
  'artist_name_norm': '60 juno',
  'listeners': 229155,
  'playcount': 1770362},
 {'artist_name_raw': 'A Day to Remember',
  'artist_name_norm': 'a day to remember',
  'listeners': 1894214,
  'playcount': 153426719},
 {'artist_name_raw': 'A Flock of Seagulls',
  'artist_name_norm': 'a flock of seagulls',
  'listeners': 1222697,
  'playcount': 12900640}]

In [ ]:
# ---------------------------------------
# EXTRACCION CHILL
# ---------------------------------------
# 1) Leemos el CSV de Spotify (una columna, sin cabecera)
#    -> devuelve lista normalizada y sin duplicados
ruta_csv_chill = "spotify_artists_chill_2019_2023.csv"
lista_chill = leer_artistas_csv(ruta_csv_chill)

print("Artistas CHILL (sin duplicados):", len(lista_chill))

# 2) Pedimos a Last.fm listeners y playcount para cada artista
resultados_chill = obtener_info_artistas(
    lista_chill,
    pausa_segundos=0.25,
    autocorrect=1,
    timeout=30,
    imprimir_cada=25
)

# 3) Guardamos el resultado en un CSV final (para no repetir llamadas)
guardar_resultados_csv(resultados_chill, "lastfm_info_artistas_chill.csv", id_genre=3)

# 4) Ver los 5 primeros
resultados_chill[:5]

Artistas CHILL (sin duplicados): 166
✅ 25/166 completados | 30s
✅ 50/166 completados | 54s
✅ 75/166 completados | 79s
✅ 100/166 completados | 102s
✅ 125/166 completados | 144s
✅ 150/166 completados | 168s


[{'artist_name_raw': '2illusions',
  'artist_name_norm': '2illusions',
  'listeners': 2479,
  'playcount': 22948},
 {'artist_name_raw': '351 Lake Shore Drive',
  'artist_name_norm': '351 lake shore drive',
  'listeners': 16886,
  'playcount': 94037},
 {'artist_name_raw': '4 Wings',
  'artist_name_norm': '4 wings',
  'listeners': 61700,
  'playcount': 228364},
 {'artist_name_raw': 'aint',
  'artist_name_norm': 'aint',
  'listeners': 35015,
  'playcount': 259327},
 {'artist_name_raw': 'ASP Project',
  'artist_name_norm': 'asp project',
  'listeners': 984,
  'playcount': 3402}]

In [ ]:
# ---------------------------------------
# EXTRACCION LATIN
# ---------------------------------------
# 1) Leemos el CSV de Spotify (una columna, sin cabecera)
#    -> devuelve lista normalizada y sin duplicados
ruta_csv_latin = "spotify_artists_latin_2019_2023.csv"
lista_latin = leer_artistas_csv(ruta_csv_latin)

print("Artistas LATIN (sin duplicados):", len(lista_latin))

# 2) Pedimos a Last.fm listeners y playcount para cada artista
resultados_latin = obtener_info_artistas(
    lista_latin,
    pausa_segundos=0.25,
    autocorrect=1,
    timeout=30,
    imprimir_cada=25
)

# 3) Guardamos el resultado en un CSV final (para no repetir llamadas)
guardar_resultados_csv(resultados_latin, "lastfm_info_artistas_latin.csv", id_genre=4)

# 4) Ver los 5 primeros
resultados_latin[:5]

Artistas LATIN (sin duplicados): 265
✅ 25/265 completados | 32s
✅ 50/265 completados | 53s
✅ 75/265 completados | 75s
✅ 100/265 completados | 119s
✅ 125/265 completados | 149s
✅ 150/265 completados | 174s
✅ 175/265 completados | 197s
✅ 200/265 completados | 217s
✅ 225/265 completados | 241s
✅ 250/265 completados | 275s


[{'artist_name_raw': 'Abraham Mateo',
  'artist_name_norm': 'abraham mateo',
  'listeners': 162950,
  'playcount': 2581795},
 {'artist_name_raw': 'Aissa',
  'artist_name_norm': 'aissa',
  'listeners': 31336,
  'playcount': 607475},
 {'artist_name_raw': 'Aitana',
  'artist_name_norm': 'aitana',
  'listeners': 225475,
  'playcount': 13481806},
 {'artist_name_raw': 'Alejandro Sanz',
  'artist_name_norm': 'alejandro sanz',
  'listeners': 671211,
  'playcount': 14460688},
 {'artist_name_raw': 'Alejo',
  'artist_name_norm': 'alejo',
  'listeners': 147663,
  'playcount': 3569689}]